# Ensemble Learning

## Initial Imports

In [69]:
import warnings
warnings.filterwarnings('ignore')

In [70]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [71]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [72]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [73]:
# Preview the data
df.tail()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
68812,10000.0,0.1502,346.76,RENT,26000.0,Source Verified,Jan-2019,low_risk,n,9.60,...,80.0,0.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0,N,N
68813,12000.0,0.2727,368.37,RENT,63000.0,Not Verified,Jan-2019,low_risk,n,29.07,...,96.2,0.0,0.0,0.0,87939.0,60350.0,13500.0,62939.0,N,N
68814,5000.0,0.1992,185.62,MORTGAGE,52000.0,Source Verified,Jan-2019,low_risk,n,14.86,...,100.0,0.0,1.0,0.0,30592.0,18611.0,3600.0,18492.0,N,N
68815,40000.0,0.0646,1225.24,MORTGAGE,520000.0,Verified,Jan-2019,low_risk,n,9.96,...,98.2,12.5,0.0,0.0,1033574.0,95958.0,100800.0,78634.0,N,N
68816,16000.0,0.1131,350.36,MORTGAGE,72000.0,Verified,Jan-2019,low_risk,n,7.02,...,94.3,0.0,1.0,0.0,251486.0,74835.0,23000.0,63090.0,N,N


In [74]:
#df = pd.DataFrame()

#df['year'] = pd.DatetimeIndex(df['issue_d']).year
#df['month'] = pd.DatetimeIndex(df['issue_d']).month

#df.head()

In [75]:
#df=df.rename(columns={'year': 'Year_issued_d', 'month': 'month_issued_d'})
#df.head()

In [76]:
#df['year'] = pd.DatetimeIndex(df['next_pymnt_d']).year
#df['month'] = pd.DatetimeIndex(df['next_pymnt_d']).month
#df=df.rename(columns={'year': 'Year_next_pymnt_d', 'month': 'month_next_pymnt_d'})
#df.head()

In [77]:
#df = df.drop(columns="issue_d")
#df = df.drop(columns="next_pymnt_d")
#df.head()

## Split the Data into Training and Testing

In [78]:
# Create our features
X = df.drop(columns='loan_status')

# Create our target
y = df['loan_status']


In [79]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [80]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [81]:
# Binary encoding using Pandas (home_ownership,verification_status,loan_status,pymnt_plan, hardship_flag, debt_settlement_flag,  columns) 
# Another approach use get_dummies transform in 1 and 0 the data to 'eliminate' bias 
X = pd.get_dummies(X)  
X.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,1,0,1,0,1,1,0,1,1


In [82]:
X.tail()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
68812,10000.0,0.1502,346.76,26000.0,9.60,0.0,0.0,9.0,0.0,2684.0,...,0,1,0,1,0,1,1,0,1,1
68813,12000.0,0.2727,368.37,63000.0,29.07,0.0,0.0,8.0,0.0,13314.0,...,0,1,0,1,0,1,1,0,1,1
68814,5000.0,0.1992,185.62,52000.0,14.86,0.0,0.0,5.0,1.0,3715.0,...,0,1,0,1,0,1,1,0,1,1
68815,40000.0,0.0646,1225.24,520000.0,9.96,0.0,1.0,21.0,0.0,59529.0,...,0,1,1,0,0,1,1,0,1,1
68816,16000.0,0.1131,350.36,72000.0,7.02,2.0,0.0,12.0,1.0,11882.0,...,0,1,0,1,0,1,1,0,1,1


## Integer Encoding

### Manual Integer Encoding

Perform a manual integer encoding of the `Month` column, use a dictionary to map months names with their corresponding numerical value.

In [83]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)
X_train.shape

(51612, 95)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [84]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler


StandardScaler()

In [85]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)
X_scaler

StandardScaler()

In [86]:
# Scale the training and testing data
X_train_scaler = scaler.transform(X_train)
X_train_scaler

array([[-1.46833559, -0.82153497, -1.49317685, ..., -0.40374336,
         0.        ,  0.        ],
       [-0.7493051 , -1.08086564, -0.69775565, ..., -0.40374336,
         0.        ,  0.        ],
       [-0.65213882, -0.50618887, -0.92691543, ..., -0.40374336,
         0.        ,  0.        ],
       ...,
       [ 0.319524  , -0.201216  , -0.133886  , ...,  2.47682094,
         0.        ,  0.        ],
       [-0.49424361,  0.35271432, -0.72156941, ..., -0.40374336,
         0.        ,  0.        ],
       [ 0.319524  , -0.50618887, -0.18477188, ...,  2.47682094,
         0.        ,  0.        ]])

In [87]:
X_test_scaler = scaler.transform(X_test)
X_test_scaler

array([[-0.65213882,  0.06433861, -0.49975826, ..., -0.40374336,
         0.        ,  0.        ],
       [-0.62541809,  0.69918009, -0.80025534, ..., -0.40374336,
         0.        ,  0.        ],
       [ 2.26284965, -0.95016298,  2.68802733, ..., -0.40374336,
         0.        ,  0.        ],
       ...,
       [-0.16630741,  0.58092531, -0.4164621 , ..., -0.40374336,
         0.        ,  0.        ],
       [-1.07967046, -0.30287362, -1.03066301, ..., -0.40374336,
         0.        ,  0.        ],
       [ 2.26284965,  0.35271432,  3.10152711, ...,  2.47682094,
         0.        ,  0.        ]])

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [88]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf_model= brf_model.fit(X_train_scaler, y_train)
predictions = brf_model.predict(X_test_scaler)

In [89]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [90]:
# Calculated the balanced accuracy score
brf_acc_score = accuracy_score(y_test, predictions)
brf_acc_score

0.8735251380412671

In [91]:
# Display the confusion matrix
brf_cm = confusion_matrix(y_test, predictions)
brf_cm_df = pd.DataFrame(
    brf_cm, index=["Actual 0", "Actual 1"], columns=["brf_Predicted 0", "brf_Predicted 1"])
print()
print("brf_Confusion Matrix")
print()
display(brf_cm_df)


brf_Confusion Matrix



,brf_Predicted 0,brf_Predicted 1
Actual 0,71,30
Actual 1,2146,14958


In [92]:
# Print the imbalanced classification report

print()
print(f"brf_Accuracy Score : {brf_acc_score}")
print()
print()
print("brf_Classification Report")
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))


brf_Accuracy Score : 0.8735251380412671


brf_Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.70      0.87      0.06      0.78      0.60       101
   low_risk       1.00      0.87      0.70      0.93      0.78      0.63     17104

avg / total       0.99      0.87      0.70      0.93      0.78      0.63     17205



In [93]:
# List the features sorted in descending order by feature importance
importances = brf_model.feature_importances_
importances_sorted = sorted(zip(brf_model.feature_importances_, X.columns), reverse=True)
importances_sorted

[(0.07876809003486353, 'total_rec_prncp'),
 (0.05883806887524815, 'total_pymnt'),
 (0.05625613759225244, 'total_pymnt_inv'),
 (0.05355513093134745, 'total_rec_int'),
 (0.0500331813446525, 'last_pymnt_amnt'),
 (0.02966959508700077, 'int_rate'),
 (0.021129125328012987, 'issue_d_Jan-2019'),
 (0.01980242888931366, 'installment'),
 (0.01747062730041245, 'dti'),
 (0.016858293184471483, 'out_prncp_inv'),
 (0.01641297102011915, 'total_bal_ex_mort'),
 (0.015220714904737209, 'mths_since_recent_inq'),
 (0.015115240704562424, 'issue_d_Mar-2019'),
 (0.014926655663448373, 'revol_bal'),
 (0.014899352873994727, 'max_bal_bc'),
 (0.014881069023035237, 'out_prncp'),
 (0.014859446582326507, 'mo_sin_old_rev_tl_op'),
 (0.014832564501144122, 'annual_inc'),
 (0.014613819728800227, 'bc_util'),
 (0.014487685026878092, 'tot_hi_cred_lim'),
 (0.013921085423763812, 'mo_sin_old_il_acct'),
 (0.013534131593418711, 'mths_since_rcnt_il'),
 (0.013364759441576994, 'total_rev_hi_lim'),
 (0.01332289882475225, 'avg_cur_bal')

### Easy Ensemble Classifier

In [64]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec_model= eec_model.fit(X_train_scaler, y_train)
eec_predictions = eec_model.predict(X_test_scaler)

In [68]:
# Calculated the balanced accuracy score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

eec_acc_score = accuracy_score(y_test, eec_predictions)
eec_acc_score

0.9422842197035746

In [95]:
# Display the confusion matrix
eec_cm = confusion_matrix(y_test, eec_predictions)
eec_cm_df = pd.DataFrame(
    eec_cm, index=["Actual 0", "Actual 1"], columns=["rf_Predicted 0", "rf_Predicted 1"]
)
print("RF-Confusion Matrix")
print
display(eec_cm_df)

RF-Confusion Matrix


,rf_Predicted 0,rf_Predicted 1
Actual 0,93,8
Actual 1,985,16119


In [96]:
# Print the imbalanced classification report
print()
print(f"EEC-Accuracy Score : {eec_acc_score}")
print()
print()
print("EEC-Classification Report:")
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, eec_predictions))


EEC-Accuracy Score : 0.9422842197035746


EEC-Classification Report:
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.92      0.94      0.16      0.93      0.87       101
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205



### Final Questions

##### 1. Which model had the best balanced accuracy score?

    Balanced accuracy is a better metric to use with imbalanced data. It accounts for both the positive and negative outcome classes and doesn’t mislead with imbalanced data.

    * In this excecise Easy Ensemble Classifier model has the best balanced accuracy score.
    
    EEC-Accuracy Score : 0.9422842197035746 > brf_Accuracy Score : 0.8735251380412671


##### 2. Which model had the best recall score?

    Recall is important to know when you really want to correctly predict the cases in the true class. The best possible recall is 1 and the worst possible is 0.

    *  In this excecise Easy Ensemble Classifier model has the best recall score.
    
        EEC-Classification  Recall: high_risk = 0.92, low_risk = 0.94, avg / total = 0.94.
      
        brf_Classification  Recal: high_risk = 0.70, low_risk = 0.87, avg / total = 0.87.
                  


##### 3. Which model had the best geometric mean score?

     Geometric mean is the combination of Sensitivity and Specificity into a single score balance. For imbalanced classification, the sensitivity might be more interesting than the specificity. Sensitivity refers to the true positive rate and summarizes how well the positive class was predicted. Specificity is the complement to sensitivity, or the true negative rate, and summarises how well the negative class was predicted.

    *  In this excercise Easy Ensemble Classifier model has the best geometric mean score.
    
        EEC-Classification  geometric mean: high_risk = 0.93, low_risk = 0.93, avg / total = 0.93.
      
        brf_Classification  geometric mean: high_risk = 0.78, low_risk = 0.78, avg / total = 0.78.

##### 4. What are the top three features?

    * The top three features are:
    
         1.- total_rec_prncp = 0.07876809003486353
         
         2.- total_pymnt = 0.05883806887524815
         
         3.- total_pymnt_inv = 0.05625613759225244
  